In [ ]:
!pip install -q openai

In [ ]:
import openai
from openai import OpenAI,OpenAIError
import pandas as pd
import os
os.environ["OPENAI_API_KEY"] = "MyAPI키입력"

In [ ]:
import openai
import pandas as pd
import os
import re

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def split_response(response):
    # 줄바꿈 한 번 또는 두 번을 기준으로 분리
    parts = re.split(r'\n\n|\n', response, maxsplit=1)
    if len(parts) > 1:
        response_part = parts[0].strip()
        reason_part = parts[1].strip()
    else:
        response_part = response.strip()
        reason_part = ""

    return response_part, reason_part

def evaluate_sentences(messages):
    response = client.chat.completions.create(
        model="gpt모델이름",  # gpt모델이름입력
        messages=messages,
        max_tokens=200,
        n=1,
        stop=None,
        temperature=1,
    )
    return response.choices[0].message.content.strip()



def run_experiment(input_file, output_file):
    # 엑셀 파일에서 문장 읽기
    df = pd.read_excel(input_file)

    results = []

    for index, row in df.iterrows():
        sentence = row['sentence']

        # 공통 질문
        common_prompt = (
            "당신은 한국어를 모국어로 하는 화자이다. 주어진 문장을 읽고 그 문장의 수용성을 아래의 척도를 사용하여 응답하라. "
            "여기서 수용성(acceptability)이란 특정 언어 표현이나 문장이 그 언어의 화자들에 의해 자연스럽고 올바른 것으로 받아들여지는 정도를 의미한다."
            "확실하지 않더라도 문장을 읽고 척도의 번호로만 응답하라.\n\n"
            "1 = 전혀 수용 불가능함\n"
            "2 = 수용 불가능함\n"
            "3 = 보통임\n"
            "4 = 수용 가능함\n"
            "5 = 매우 수용 가능함\n"
                    )

        messages = [
            {"role": "system", "content": common_prompt},
            {"role": "user", "content": f"문장: {sentence}\n\n응답:\n\n그렇게 판단한 이유는 무엇인가?"}
        ]
        response_full = evaluate_sentences(messages)
        response, reason = split_response(response_full)

        # 결과 저장
        results.append({
            "id": row['id'],
            "sentence": sentence,
            "gold_label": row['gold_label'],
            "응답": response,
            "이유": reason,

        })

    # 결과를 데이터프레임으로 변환
    results_df = pd.DataFrame(results)

    # 결과를 엑셀 파일로 저장
    results_df.to_excel(output_file, index=False)

# 실험 실행
input_file = '입력파일경로.xlsx'  # 입력 엑셀 파일 경로
output_file = '출력파일경로.xlsx'  # 출력 엑셀 파일 경로
run_experiment(input_file, output_file)